**What dataset are we working with?**

https://www.kaggle.com/datasets/mathurinache/citation-network-dataset/data?select=dblp.v12.json

How was the dataset collected?

For what purpose was the dataset collected?

To grab our dataset, we need to set up our env to with the kaggle api.

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
# Ensure the .kaggle directory exists
!mkdir -p ~/.kaggle

# Copy the kaggle_api.json from Google Drive to the .kaggle directory and rename it to kaggle.json
!cp "/content/drive/Shareddrives/COMP 440 S23: Nostradami/kaggle.json" ~/.kaggle/kaggle.json

# Set the permissions of the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json




Mounted at /content/drive/


In [2]:
!kaggle datasets download --force -d mathurinache/citation-network-dataset

 99% 4.17G/4.19G [00:29<00:00, 235MB/s]
100% 4.19G/4.19G [00:29<00:00, 151MB/s]


In [3]:
!pip install ijson

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 2.2 MB/s eta 0:00:00


In [ ]:
# import zipfile
# import ijson

# # Specify the path to the ZIP file you've downloaded
# zip_file_path = '/content/citation-network-dataset.zip'

# # Specify the filename of the JSON file within the ZIP archive
# # Adjust this filename based on the actual name of the JSON file inside the ZIP
# json_file_name = 'dblp.v12.json'

# # Use the zipfile library to open the ZIP file
# with zipfile.ZipFile(zip_file_path, 'r') as z:
#     # Extract the specific JSON file from the ZIP into memory
#     with z.open(json_file_name) as json_file:
#         # Now, json_file acts like an open file object for the JSON file
#         # You can use ijson to parse the JSON file incrementally here

#         # Example: Iterate over items in the JSON file
#         objects = ijson.items(json_file, 'item')
#         for obj in objects:
#             #print(obj)


In [60]:
import pandas as pd
import zipfile
import ijson
import os

# Initialize lists to store the data
ids, titles, years, n_citations, doc_types, authors_list, venues, fos = [], [], [], [], [], [], [], []

# Define a function to process a single JSON object
def process_json_obj(obj):
    ids.append(obj.get('id'))
    titles.append(obj.get('title'))
    years.append(obj.get('year'))
    n_citations.append(obj.get('n_citation'))
    doc_types.append(obj.get('doc_type'))
    authors_names = ", ".join([author.get('name') for author in obj.get('authors', [])])
    authors_list.append(authors_names)
    venues.append(obj.get('venue', {}).get('raw', ''))
    tmp = []
    for field in obj.get("fos", []):
        tmp.append(field.get("name"))
    fos.append(', '.join(map(str, tmp)))


# Specify the maximum number of objects to process
MAX_OBJECTS = 1

# Open the ZIP file and process the JSON file within
with zipfile.ZipFile('/content/citation-network-dataset.zip', 'r') as z:
    with z.open('dblp.v12.json') as json_file:  # Adjust the path if necessary
        objects = ijson.items(json_file, 'item')
        for i, obj in enumerate(objects):
            if i >= MAX_OBJECTS:
                break
            process_json_obj(obj)
            print(fos)

# Create a DataFrame from the collected data
citations_df = pd.DataFrame({
    'ID': ids,
    'Title': titles,
    'Year': years,
    'Citations': n_citations,
    'Document Type': doc_types,
    'Authors': authors_list,
    'Venue': venues,
    'Field of Study': fos
})

with open("data.csv", "x") as csv:
  csv.write(citations_df.to_csv())

# Display basic information about the DataFrame to ensure it's loaded correctly
print(citations_df.info())
print(citations_df.head())

['Telecommunications network, Computer science, Mind map, Human–computer interaction, Multimedia, Empirical research, Comprehension, Communications protocol']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              1 non-null      int64 
 1   Title           1 non-null      object
 2   Year            1 non-null      int64 
 3   Citations       1 non-null      int64 
 4   Document Type   1 non-null      object
 5   Authors         1 non-null      object
 6   Venue           1 non-null      object
 7   Field of Study  1 non-null      object
dtypes: int64(3), object(5)
memory usage: 192.0+ bytes
None
     ID                                              Title  Year  Citations  \
0  1091  Preliminary Design of a Network Protocol Learn...  2013          1   

  Document Type                                            Authors  \
0    Confere

In [ ]:
citations_df.head()

In [ ]:
citations_df.columns

Index(['ID', 'Title', 'Year', 'Citations', 'Document Type', 'Authors',
       'Venue'],
      dtype='object')

In [ ]:
# Trim out the rows with n_citations = 0
print(citations_df.shape)

There are 4894081 rows of data.

In [ ]:
# These columns are Series data types
# print(citations_df.Year.min())
# print(citations_df.Year.max())

# Look for papers whose year isn't 0
years_citations_df = citations_df[citations_df["Year"] != 0]
print(years_citations_df.shape)
print(years_citations_df.Year.min())
print(years_citations_df.Year.max())

There are 9 papers whose Year is 0.

In [ ]:
# Papers written between 1800 (inclusive) and 1899 (inclusive)
print(years_citations_df[(years_citations_df["Year"] >= 1800) & (years_citations_df["Year"] < 1900)])

In [ ]:
# Papers written between 1900 (inclusive) and 1999 (inclusive)
print(years_citations_df[(years_citations_df["Year"] >= 1900) & (years_citations_df["Year"] < 1999)])

In [ ]:
# Papers written past 2000 (inclusive)
print(years_citations_df[(years_citations_df["Year"] >= 2000)])

In [ ]:
# testingData = {
#     'Name': ['John', 'Doe', 'Alice', 'Bob', 'Chris'],
#     'Age': [25, 30, 22, 28, 35],
#     'Salary': [50000, 60000, None, 75000, 90000],
#     'Experience': [2, 5, 1, None, 10]
# }

# testing_df = pd.DataFrame(testingData)

# print(testing_df["Salary"].isna())
# print(testing_df["Salary"].isna().sum())

0    False
1    False
2     True
3    False
4    False
Name: Salary, dtype: bool
1


In [ ]:
print(citations_df["ID"].isna().sum())
print(citations_df["Title"].isna().sum())

print(len(citations_df.Title.unique()))
print("There are " + str(citations_df.shape[0] - len(citations_df.Title.unique())) + " papers that share the same title" )

0
0
4696970
There are 197111 papers that share the same title


Of the 4894081 papers in the dataset, 197111 papers share the same title.

In [ ]:
print(citations_df.Citations.min())
print(citations_df.Citations.max())

print(citations_df[citations_df["Citations"] >= 48327])
print(citations_df[citations_df["Citations"] == 0])

0
48327
                 ID                                     Title  Year  \
4696136  2041404167  The Mathematical Theory of Communication  1949   

         Citations Document Type        Authors Venue  
4696136      48327          Book  C. E. Shannon        
                ID                                              Title  Year  \
4             5411  COMPARING GNG3D AND QUADRIC ERROR METRICS METH...  2009   
6             6522  Improved Secret Image Sharing Method By Encodi...  2011   
7             6762  A Self-Stabilizing Algorithm for Finding the C...  2003   
8             8373  Formal agent-oriented ubiquitous computing: a ...  2012   
9             8763  Fur Visualisation for Computer Game Engines an...  2014   
...            ...                                                ...   ...   
4894070  999760546  Trust, Privacy and Security in Digital Busines...  2011   
4894075  998513720       Communication: Words and Conceptual Systems.  2015   
4894077   99884553   Blind